In [1]:
import numpy as np
import h5py
import sys


sys.path.append(".")
dirname = "./bgw_files/k-4x4x4/"

In [2]:
# epsinp and sigmain data.
from qtm.interfaces.bgw.epsinp import Epsinp
epsinp = Epsinp.from_epsilon_inp(filename=dirname+'epsilon.inp')

from qtm.interfaces.bgw.sigmainp import Sigmainp
sigmainp = Sigmainp.from_sigma_inp(filename=dirname+'sigma.inp')

In [3]:
from qtm.interfaces.bgw import inp
from qtm.interfaces.bgw.wfn2py import wfn2py

# WFN data
wfndata = wfn2py(dirname+'WFN.h5')
wfnqdata = wfn2py(dirname+'WFNq.h5')

# RHO data
rho = inp.read_rho(dirname+"RHO")

# Vxc data
vxc = inp.read_vxc(dirname+"vxc.dat")

In [4]:
from qtm.gw.core import QPoints
from qtm.gw.epsilon import Epsilon

epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)

Vcoul calculation for qpts: 100%|██████████| 64/64 [00:00<00:00, 19623.91it/s]


In [5]:
from tqdm import trange
from qtm.gw.core import reorder_2d_matrix_sorted_gvecs, sort_cryst_like_BGW


def calculate_epsilon(numq=None, writing=False):
    epsmats = []
    if numq is None:
        numq = epsilon.qpts.numq

    for i_q in trange(0, numq, desc="Epsilon> q-pt index"):
        # Create map between BGW's sorting order and QTm's sorting order
        gkspc = epsilon.l_gq[i_q]
        
        if i_q == epsilon.qpts.index_q0:
            key = gkspc.g_norm2
        else:
            key = gkspc.gk_norm2

        indices_gspace_sorted = sort_cryst_like_BGW(
            cryst=gkspc.g_cryst, key_array=key
        )

        # Calculate matrix elements
        M = next(epsilon.matrix_elements(i_q=i_q))

        # Calculate polarizability matrix (faster, but not memory-efficient)
        chimat = epsilon.polarizability(M)

        # Calculate polarizability matrix (memory-efficient)
        # chimat = epsilon.polarizability_active(i_q)

        # Calculate epsilon inverse matrix
        epsinv = epsilon.epsilon_inverse(i_q=i_q, polarizability_matrix=chimat, store=True)


        epsinv = reorder_2d_matrix_sorted_gvecs(epsinv, indices_gspace_sorted)
        epsilon.l_epsinv[i_q] = epsinv
        
        # Compare the results with BGW's results
        if i_q == epsilon.qpts.index_q0:
            epsref = epsilon.read_epsmat(dirname + "eps0mat.h5")[0][0, 0]
            if writing:
                epsilon.write_epsmat(
                    filename="test/epsilon/eps0mat_qtm.h5", epsinvmats=[epsinv]
                )
        else:
            epsref = np.array(epsilon.read_epsmat(dirname + "epsmat.h5")[i_q - 1][0, 0])
            epsmats.append(epsinv)

        # Calculate stddev between reference and calculated epsinv matrices
        std_eps = np.std(epsref - epsinv) / np.sqrt(np.prod(list(epsinv.shape)))

        epstol = 1e-16
        if np.abs(std_eps) > epstol:
            print(f"Standard deviation exceeded {epstol} tolerance: {std_eps}, for i_q:{i_q}")
            print(epsref[:2,:2])
            print(epsinv[:2,:2])

    if writing:
        epsilon.write_epsmat(filename="test/epsilon/epsmat_qtm.h5", epsinvmats=epsmats)


epsinp.no_min_fftgrid = True
epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)
calculate_epsilon()

Epsilon> q-pt index: 100%|██████████| 64/64 [00:20<00:00,  3.05it/s]


In [6]:
from qtm.gw.sigma import Sigma

outdir = dirname+"temp/"

sigma = Sigma.from_data(
    wfndata=wfndata,
    wfnqdata=wfnqdata,
    sigmainp=sigmainp,
    epsinp=epsinp,
    l_epsmats=epsilon.l_epsinv,
    rho=rho,
    vxc=vxc,
    outdir=outdir,
)

# sigma = Sigma(
#     crystal = wfndata.crystal,
#     gspace = wfndata.grho,
#     kpts = wfndata.kpts,
#     kptsq = wfnqdata.kpts,
#     l_wfn = wfndata.l_wfn,
#     l_wfnq = wfnqdata.l_wfn,
#     l_gsp_wfn =  wfndata.l_gk,
#     l_gsp_wfnq = wfnqdata.l_gk,
#     qpts = QPoints.from_cryst(wfndata.kpts.recilat, epsinp.is_q0, *epsinp.qpts),
#     epsinp = epsinp,
#     sigmainp = sigmainp,
#     l_epsmats = epsilon.l_epsinv,
#     rho = rho,
#     vxc = vxc,
#     outdir = outdir,
# )

Vcoul calculation for qpts: 100%|██████████| 64/64 [00:00<00:00, 18958.65it/s]


vcoul: Vcoul:
        * gspace = <qtm.gspace.gspc.GSpace object at 0x7b9261896590>
        * qpts = <qtm.gw.core.QPoints object at 0x7b924fea5c50>
        * bare_coulomb_cutoff = 10.0
        * avgcut = 1e-05
        * l_gspace_q = <class 'list'> of length 64
        * vcoul = <class 'list'> of length 64
        * N_SAMPLES = 2500000.0
        * N_SAMPLES_COARSE = 250000.0
        * SEED = 5000
        


Vcoul calculation for qpts: 100%|██████████| 64/64 [00:43<00:00,  1.47it/s]


In [7]:
from kernelv2 import KernelMtxElv2
KernelClass = KernelMtxElv2.from_BGW(
    wfndata=wfndata,
    wfnqdata=wfnqdata,
    epsinp=epsinp,
    sigmainp=sigmainp,
    vcoul=sigma.vcoul,
    l_epsmats=epsilon.l_epsinv,
    parallel=True,
)

In [8]:
numq = KernelClass.qpts.numq
numk = KernelClass.kpts.numk

In [9]:
from interpolatev2 import InterpMtxElv2

InterpClass = InterpMtxElv2.from_BGW(
    wfn_finedata=wfndata,
    wfn_coarsedata=wfndata,
    epsinp=epsinp,
    sigmainp=sigmainp,
    kernel=KernelClass,
    parallel=True,
)

In [10]:
def print_2d_matrix(matrix, decimals=3):
    for row in matrix:
        print(" ".join(f"{val:9.{decimals}f}" for val in row))

In [11]:
ikf, ikc = np.random.randint(0, numk, size=2)
print(f"ikf, ikc = {ikf}, {ikc}")

ikf, ikc = 58, 23


In [12]:
coeff_val = InterpClass.coeff_mtxel(fine_k_idx=ikf, coarse_k_idx=ikc, type="val")
coeff_con = InterpClass.coeff_mtxel(fine_k_idx=ikf, coarse_k_idx=ikc, type="con")

In [13]:
print(f"coeff_val is")
print_2d_matrix(coeff_val, decimals=3)

print(f"\n coeff_37_con is")
print_2d_matrix(coeff_con, decimals=3)

coeff_val is
-0.076+0.203j 0.304-0.788j -0.488-0.005j 0.045+0.009j
0.266+0.346j -0.223-0.296j 0.424-0.665j -0.153+0.165j
0.487-0.208j -0.265+0.109j -0.352-0.378j -0.330-0.512j
0.258+0.412j 0.184+0.302j -0.009+0.012j -0.596+0.532j

 coeff_37_con is
-0.646+0.619j -0.001-0.001j -0.348+0.278j 0.040+0.008j
-0.139+0.264j 0.073+0.028j 0.364-0.563j 0.671-0.078j
-0.902-0.191j -0.030+0.329j 0.014+0.004j 0.076+0.184j
-0.079-0.808j -0.452+0.099j 0.002+0.273j -0.176+0.148j


In [14]:
from kernel import KernelMtxEl

q0 = [0.001, 0.001, 0.001]
l_qpts = np.array(epsinp.qpts)
l_qpts[0] *= 0
qpts = QPoints.from_cryst(wfndata.kpts.recilat, None, *l_qpts)

kernelclass0 = KernelMtxEl.from_BGW(
    wfndata=wfndata,
    epsinp=epsinp,
    sigmainp=sigmainp,
    q0=q0,
    l_epsmats=epsilon.l_epsinv,
    parallel=True,
)

In [15]:
from intkernel import InterpMtxEl

InterpClass0 = InterpMtxEl.from_BGW(
    wfn_finedata=wfndata,
    wfn_coarsedata=wfndata,
    epsinp=epsinp,
    sigmainp=sigmainp,
    kernel=kernelclass0,
    parallel=True,
)

In [16]:
coeff_val0 = InterpClass0.coeff_mtxel(ikf, ikc, "val")
coeff_con0 = InterpClass0.coeff_mtxel(ikf, ikc, "con")


print(f"coeff_val0 is")
print_2d_matrix(coeff_val0, decimals=3)

print(f"\n coeff_37_con0 is")
print_2d_matrix(coeff_con0, decimals=3)

coeff_val0 is
0.169+0.276j -0.008+0.011j -0.545+0.487j -0.062-0.597j
-0.286+0.118j -0.380-0.408j -0.357-0.554j 0.399+0.021j
-0.247-0.329j 0.471-0.738j -0.169+0.183j -0.009-0.043j
0.304-0.787j -0.487-0.005j 0.045+0.009j 0.159-0.151j

 coeff_37_con0 is
-0.646+0.619j -0.001-0.001j -0.348+0.278j 0.040+0.008j
-0.139+0.264j 0.073+0.028j 0.364-0.563j 0.671-0.078j
-0.902-0.191j -0.030+0.329j 0.014+0.004j 0.076+0.184j
-0.079-0.808j -0.452+0.099j 0.002+0.273j -0.176+0.148j
